In [1]:
logFileName = "car_log.txt"

In [2]:
class Logger:
    def __init__(self, road, interval):
        self.cars = list()
        self.road = road
        self.lightInterval = interval
    
    def addCar(self, car):
        self.cars.append(car)
    
    def outputToFile(self):
        f = open(logFileName, "w")
        
        # Log road
        f.write("Road" + " " + str(self.road.lanes[0].length) + " " + str(len(self.road.lanes)*2.5) + "\n")
#         f.write(self.road.name + " " + self.road.lanes[0].length + " " + len(self.road.lanes)*2.5)
        
        # Log lights
        f.write(str(self.lightInterval) + "\n")
    
        # Log cars with locations
        for car in self.cars:
            locationStrList = list()
            for loc in car.location_log:
                locationStrList.append(str(loc[0]) + "," + str(loc[1]))
            
            line = str(car.car_id) + " "      
            line += " ".join(locationStrList) + "\n"
            f.write(line)
            
        f.close()

In [3]:
class Vehicle:
    types = ["Sedan", "Truck", "Motorcycle"]
    # Super-class of car, TBI (to be implemented)

In [4]:
class Lane:    
    def __init__(self, length, width, position):
        self.parent_road = None
        self.length = length
        self.width = width
        self.position = position
        self.cars = []

    def __str__(self):
        strprt = ""
        for car in self.cars:
            strprt += car + ":" + str(car.location) + "  "
        return strprt
    
    def addToLane(self, car):
        self.cars.append(car)
        car.set_lane(self)
    
    def set_road(self, road):
        self.parent_road = road
        
    def remove_car(self, car):
        self.cars.remove(car)
        
    def isEmpty(self):
        return True if not self.cars else False
    
#  I think we need only one list containing car objects.
#  I converted the static variables to member variables

In [5]:
class Road:
    
    def __init__(self, *lanes):
        self.lanes = list(lanes)
        self.length = lanes[0].length
        self.collided_cars = []
    
    @classmethod
    def generateLanes(self, laneCount):
        for i in range(0,laneCount):
            self.lanes.append(Lane(laneLength, self))
    
    def isEmpty(self):
        for lane in self.lanes:
            if not lane.isEmpty():
                return False
        return True
    
# Road is implemented a little bit

In [6]:
class Car:
    max_speed = 10.0
    min_speed = 0
    max_acc = 0.5
    max_brake = 2
    size = 2.0
    
    def __init__(self, car_id, car_type, driver, initialSpeed=0.0, initialLocation=[0.0, 0.0]):
        self.car_id = car_id
        self.car_type = car_type
        self.speed = initialSpeed
        self.location = initialLocation
        self.location_log = list()
        self.logPosition()
        self.driver = driver
        self.combo = 0.0 # This variable is used to simulate smooth braking
        self.lane = None
        self.is_collided = False
        self.mode = 0
            
    def set_lane(self, lane):
        self.lane = lane
        
    def accelerate(self):
        self.combo = 0  # Reset the brake combo
        self.speed += self.max_acc
        if self.speed > self.max_speed:
            self.speed = self.max_speed
        self.__move()
        
    def decelerate(self):
        dec_coef = 0.0
        combo_increment = 0.0
        dtype = self.driver.type
        if dtype == "Rookie":
            dec_coef = 0.9
            combo_increment = 0.8
        elif dtype == "Hasty":
            dec_coef = 0.1
            combo_increment = 0.05
        elif dtype == "Professional":
            dec_coef = 0.5
            combo_increment = 0.2
            
        speed_reduction = dec_coef * Car.max_brake + self.combo
        self.combo += combo_increment
        if speed_reduction > Car.max_brake:
            speed_reduction = Car.max_brake
        self.speed -= speed_reduction
        
        if self.speed < self.min_speed:
            self.speed = self.min_speed
            self.combo = 0
        self.__move()
        
    def __move(self):
        self.location[0] += self.speed
        self.logPosition()
        
    def logPosition(self):
        self.location_log.append(list(self.location))
    
    def search_front_car(self):
        for car in self.lane.cars:
            if car.car_id == self.car_id:
                continue
            self_front_bumper = self.location[0] + Car.size / 2
            car_back_bumper = car.location[0] - Car.size / 2
            if self_front_bumper < car_back_bumper <= self_front_bumper + self.driver.vision and self.calc_relative_speed(car) > 0: 
                return car
        return None
    
    def calc_relative_speed(self, car):
        return self.speed - car.speed
    
    def check_mirrors(self):       
        selected_cars = None
        if self.lane.position == "right":
            selected_cars = self.lane.parent_road.lanes[0].cars
        else:
            selected_cars = self.lane.parent_road.lanes[1].cars
            
        upper_limit_x = self.location[0] + Car.size / 2
        lower_limit_x = self.location[0] - Car.size / 2 - self.driver.vision
        
        for car in selected_cars:
            if lower_limit_x <= car.location[0] + Car.size / 2 <= upper_limit_x:
                return True
        return False
        
    def decide(self):
        self.check_collision()
        if self.is_collided:
            self.speed = 0
        else:
            other_position = "left" if self.lane.position == "right" else "right"

            if self.mode:
                if other_position == "left":
                    if self.location[1] > -2.5:
                        self.location[1] -= 0.5
                    else:
                        self.mode = 0
                        self.lane.addToLane(self)
                        self.lane.remove_car(self)
                else:
                    if self.location[1] < 2.5:
                        self.location[1] += 0.5
                    else:
                        self.mode = 0
                        self.lane.addToLane(self)
                        self.lane.remove_car(self)
            else:
                if self.search_front_car() is not None:
                    if self.check_mirrors():
                        self.mode = 1
                    else:
                        self.decelerate()
                else:
                    self.accelerate()
                    
    def check_collision(self):
        left_boundary = self.location[1] - Car.size / 4
        right_boundary = self.location[1] + Car.size / 4
        up_boundary = self.location[0] + Car.size / 2
        down_boundary = self.location[0] - Car.size / 2
        
        current_road = self.lane.parent_road
        for lane in current_road.lanes:
            for car in lane.cars:
                if car.car_id != self.car_id and (down_boundary <= car.location[0] <= up_boundary and left_boundary <= car.location[1] <= right_boundary):
                    self.is_collided = True
                    car.is_collided = True
                    self.speed = 0
                    car.speed = 0
                    current_road.collided_cars.append(self)
                    current_road.collided_cars.append(car)
                    print("collided")
                    return
        self.is_collided = False
        print("not collided")
        
#  It could be implemented as car + driver = agent
#  Also driver types should be classes like enums or interfaces instead strings?

In [7]:
class Driver:
    
    def __init__(self, type, gender, age):
        self.type = type
        self.gender = gender
        self.age = age
        self.vision = self.__set_vision()
        
    def __set_vision(self):
        avg_vision = 40
        age_threshold = 20
        return avg_vision + age_threshold - self.age
        
    # Adds more functionality to Car class objects or vice versa, TBI

In [8]:
class SimulationEngine:
    def __init__(self, road):
        self.road = road
        
    def moveCars(self):
        removed_cars = 0
        while(not self.road.isEmpty() and removed_cars + len(self.road.collided_cars) < 20):
            for lane in self.road.lanes:
                for car in lane.cars:
                    car_on_road = len(self.road.lanes[0].cars) + len(self.road.lanes[1].cars)
                    car.decide()                    
                    if(car.location[0] > self.road.length):
                        lane.cars.remove(car)
                        removed_cars +=1
    
# Cars do not change their locations simultaneously in this implementation

In [9]:
import random

# Simulation parameters
driver_types = ["Rookie", "Professional", "Hasty"]
genders = ["Female", "Male"]
car_types = ["Sedan"]
age_range = list(range(18, 50))

# Road and lanes
myLane1 = Lane(1000, 5, "left")
myLane2 = Lane(1000, 5, "right")
my_road = Road(myLane1, myLane2)
myLane1.set_road(my_road)
myLane2.set_road(my_road)

# Setting up cars
cars = []
for i in range(1,40):
    driver_type = driver_types[random.randint(0, len(driver_types) - 1)]
    gender = genders[random.randint(0, len(genders) - 1)]
    age = age_range[random.randint(0, len(age_range) - 1)]
    car_type = car_types[random.randint(0, len(car_types) - 1)]
    driver = Driver(driver_type, gender, age)
    speed = random.random() * 10
    x_loc = float(random.randint(0, 200))
    y_loc = None
    if random.randint(0, 1):
        y_loc = 2.5
    else:
        y_loc = -2.5
    car_name = f"{car_type}{i + 1}"
    car = Car(car_name, car_type, driver, speed, [x_loc, y_loc])
    if y_loc == -2.5:
        myLane1.addToLane(car)
    else:
        myLane2.addToLane(car)
    cars.append(car)

# Simulation
mySimEngine = SimulationEngine(my_road)
mySimEngine.moveCars()

# Logging
myLogger = Logger(my_road, 50)

for car in cars:
    myLogger.addCar(car)

myLogger.outputToFile()